# ML Laboratory 03: MultiLayer Perceptron networks

## 1. Objective

Students should understand and be able use a multi-layer fully-connected networks in Matlab

## 2. Theoretical aspects

Multi-layer perceptron (fully-connected) neural networks are widely used for classificaton of small, simple datasets.

### Multilayer perceptron

#### Cascading neurons 

We start from last week's essential message: **A single neuron creates a single hyperplane and separates the input space in two categories 0 or 1)**
  - **"neuron"** = one logistic regression operation
  - **"hyperplane"** = a linear boundary surface, with dimension N-1 
  - with a smooth sigmoid transition zone between the two classes
  
What if we have a dataset as follows? How to do classification here?

<div>
<img src=img/DatasetAngled1.png align="center" width="200"/>
</div>

![How to separate the classes in this dataset? ]()


Solution: use **two neurons**: 
  
  - each one draws a hyperplane (a line)
  - aggregate their results into the final outcome: "When both neurons say 1, output class is 1. Otherwise, output class is 0".


<div>
<img src=img/DatasetAngled2.png align="center" width="400"/>
</div>



Combining the results of both neurons in the final result is **also done with a (third) neuron**. Thus, we have **cascading neurons**.


<div>
<img src=img/Network2plus1.png align="center" width="500"/>
</div>


Neurons operating on the same inputs form a **layer**. We have two layers now:

- The inputs (this does not contain neurons, just the inputs, but it is commonly named "the input layer")
- The hidden layer (middle)
- The output layer (the output neuron)

What if we want a boundary composed of 3 sides? Use three neurons in the hidden layer.

What if we want a curved boundary? Use many more neurons (approximate the curve from many lines)

**Any hypersurface** can be obtained with just two layers, provided there are enough neurons in the hidden layer:

  1. The hidden layer draws some hyperplanes (e.g. lines)
  2. The output layer combines the results into output values

#### Multiple outputs

What if we have 4 output classes?

Have 4 neurons in the output layer, one for each class. When the input belongs to class $k$, the $k$-th neuron should produce 1, and all the others should produce 0.

**One-hot encoding**: When we train the network, we need to tell it what is the desired output (target). This is known as **encoding**. 
For an input of class $k$, we tell the network to produce a vector with a single value of 1, on position $k$.
$$\begin{bmatrix} 0 \\ \vdots \\ 0 \\ 1 \\ 0 \\ \vdots \end{bmatrix}$$

After training, when running the model, we look at **the location** of the highest value and the location is the predicted class.

#### Multiple layers

We can actually have more than 2 layers in a network. We can have as many as we want! Interpretation:

   - first hidden layer draws some hyperplanes
   - next layer combines hyperplanes into some simpler shapes
   - next layer combines the simple shapes into more complex shapes
   - ....
   - final layer gives the output

In practice, it is often better to have **more layers with fewer neurons** than 2 layers but with a huge hidden layer.

However, training many layers and many neurons is **difficult**, i.e. it can overfit, become unstable, etc.

#### Matrix form

**One neuron** does a linear combination of the inputs, followed by activation function:
$$\begin{bmatrix} w_1 & x_w & \dots & w_N & b \end{bmatrix} \cdot \begin{bmatrix} x_1 \\ x_2 \\ \vdots \\ x_N \\ 1 \end{bmatrix} = z 
\rightarrow a$$

**A layer of $M$ neurons is just $M$ neurons next to each other**:
$$\begin{bmatrix} 
w_{11} & w_{12} & \dots & w_{1N} & b_1 \\
w_{21} & w_{22} & \dots & w_{2N} & b_2 \\
\vdots  & \vdots & \dots & \vdots & \vdots \\
w_{M1} & w_{M2} & \dots & w_{MN} & b_M \\
\end{bmatrix} \cdot \begin{bmatrix} x_1 \\ x_2 \\ \vdots \\ x_N \\ 1 \end{bmatrix} = 
\begin{bmatrix} 
z_1 \\ z_2 \\ \dots \\ z_M
\end{bmatrix}
\rightarrow 
\begin{bmatrix} 
a_1 \\ a_2 \\ \dots \\ a_M
\end{bmatrix}
$$

Each layer is characterized by the weight matrix $W$.

The whole network can be understood as a sequence of matrix multiplications and activation functions.

**The next layer** takes as inputs the outputs $a_i$ of the previos layer, and does the same.

### The model

The multi-layer perceptron model contains $L$ layers, each layer consisting of a matrix multiplication and activation function:

$$\begin{align}z^{1} =& W^{1} \cdot X \\
a^{1} =& activation(z^{1}) \\
\\
z^{2} =& W^{2} \cdot a^{1} \\
a^{2} =& activation(z^{2}) \\
\\ 
... \\
z^{k} =& W^{k} \cdot a^{k-1} \\
a^{k} =& activation(z^{k})\end{align}$$

Here, $W^{k}$ is a matrix and $z{k}$, $a{k}$ are vectors (columns).

The activation function can be the **sigmoid**, **ReLU**, **tanh** etc. Typically the output used sigmoid, but all others are up to the designer.

**Inputs**: 

   - a matrix $X$ with every input vector being a column (according to the equations below; we can also transpose all matrices and vectors, 
   if we want).
 
**Outputs** (assuming one-hot encoding):
 
  - a vector $\hat{y}$ which should be understood as scores/probability of belonging in each class
  - the **location of the maximum** value gives the predicted class
  

### The model parameters

The model parameters are the weight matrices $W^{k}$ of every layer. The element $w^{k}_{ij}$ is the weight in the $k$-th layer, $i$-th neuron, $j$-th input of it.

Every neuron has a bias input. We presume that the bias is included in the weight matrices,
like we did until now (e.g. like a fake input equal to 1 is appended to the input of every layer).

### The cost function

For classification, the cross-entropy is used.

For a single input:

$$L(y, \hat{y}) = - y_1 \log{\hat{y_1}} - \dots - y_n \log{\hat{y_n}} = -\log{\hat{y_{class}}},$$

where $\hat{y_{class}}$ is the model's predicted probability for the true class of the input.

For multiple inputs: do the average of all
$$J = \frac{1}{N} \sum_i L(y^i, \hat{y}^i)$$


### Training

Training is done with **backpropagation** and gradient descent (or some variant of it).

**Backpropagation** = the technique to compute the derivatives of $J$  with respect to all parameters in the network.

#### Backpropagation

Assume we have a network with 4 layers.

$$\begin{align}z^{1} =& W^{1} \cdot X \\
a^{1} =& activation(z^{1}) \\
\\
z^{2} =& W^{2} \cdot a^{1} \\
a^{2} =& activation(z^{2}) \\
\\ 
z^{3} =& W^{3} \cdot a^{2} \\
a^{3} =& activation(z^{3}) \\
\\ 
z^{4} =& W^{4} \cdot a^{3} \\
a^{4} =& activation(z^{4})\end{align}$$

The final results $a^{4}$ are the outputs $a^{4} = \hat{y}$.

Just like in logistic regression, we can compute the derivatives for the final layer, $\frac{dJ}{dW^{4}}$ and $\frac{dJ}{da^3}$

For the third layer, we compute its own derivatives, $\frac{da^{3}}{dW^{3}}$ and $\frac{da^{3}}{da^2}$. Together with the $\frac{dJ}{da^3}$ received as inputs from the above layer, we have:

$$\frac{dJ}{dW^{3}} = \frac{dJ}{da^3} \cdot \frac{da^{3}}{dW^{3}}$$ 
and 
$$\frac{dJ}{da^{2}} = \frac{dJ}{da^3} \cdot \frac{da^{3}}{da^{2}}$$ 

For the second layer, we compute its own derivatives, $\frac{da^{2}}{dW^{2}}$ and $\frac{da^{2}}{da^1}$. Together with the $\frac{dJ}{da^2}$ received as inputs from the above layer, we have:

$$\frac{dJ}{dW^{2}} = \frac{dJ}{da^2} \cdot \frac{da^{2}}{dW^{2}}$$ 
and 
$$\frac{dJ}{da^{1}} = \frac{dJ}{da^1} \cdot \frac{da^{2}}{da^{1}}$$ 

Finally, the input layer computes its own derivatives, $\frac{da^{1}}{dW^{1}}$, aand together with the $\frac{dJ}{da^{1}}$ received from the layer above, computes:
$$\frac{dJ}{dW^{1}} = \frac{dJ}{da^1} \cdot \frac{da^{1}}{dW^{1}}$$

In backpropagation, **each layer (each operation, really) does the following**:

1. Has some inputs I, parameters P, and outputs O.
2. Knows show to compute its own derivatives $\frac{dO}{dP}$ and $\frac{dO}{dI}$
3. Receives as input from the next layer the quantity $\frac{dJ}{dO}$
4. Computes $\frac{dJ}{dP} = \frac{dJ}{dO} \cdot \frac{dO}{dP}$. This will be used in Gradient Descent.
5. Computes $\frac{dJ}{dI} = \frac{dJ}{dI} \cdot \frac{dO}{dI}$ and passes them back to the preceding layer.

Backpropagation is a **computational graph** (sequence of operations) not unlike the model itself is just a sequence of operations.
The only difference is that the data travels **backwards**,  from the network output towards its input. The "data" here is the gradients (derivatives).

Training the model means repeating the two passes:

1. **Forward pass**: run the model (from the inputs, and current parameters, compute the outputs and the cost function)
2. **Backward pass**: backpropagation + gradient descent (from the cost function, compute gradients and update parameters, going backwards to the input
3. Repeat

After the gradients are calculated, we can update the parameters.

**Gradient descent** refers to the typical update rule $W = W - \mu \frac{dJ}{dW}$. There exist also some smarter variations of it.


### Matlab functions for working with neural networks

- `nprtool()` (for classification)
- `nftool()` (for regression)
- `nnstart()` or `nntool()`: entry-point for both of the above

### Example (walkthrough)

Let's do a sample classification.

Load the `wine_dataset` in Matlab and inspect the data:

- how many inputs are there?
- how many output categories?

In [ ]:
load wine_dataset

Start the `nprtool()` or the `nnstart()` and run training.

FInally, do some predictions.

# 3. Practical work

We use the same data as in linear regression, but instead we try to predict if one of the two possibilities: the quality score is <=5 (class 0) or the quality score is > 5 (class 1).

As a reminder, the data comes from here: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009, and contains 11 numerical chemical measurements for some different brands of red wines, together with a quality score indicated by buyers (quality goes from 3 to 8).

Inputs:

   - 1 - fixed acidity
   - 2 - volatile acidity
   - 3 - citric acid
   - 4 - residual sugar
   - 5 - chlorides
   - 6 - free sulfur dioxide
   - 7 - total sulfur dioxide
   - 8 - density
   - 9 - pH
   - 10 - sulphates
   - 11 - alcohol 
   
Outputs:

   - 12 - quality
   

Let's load the data first 

In [47]:
Data = readmatrix('winequality-red.csv');
X = Data(:,1:11);       % 11 columns for the inputs
N = size(Data,1);       % The number of wines in the set (1599)

Y = Data(:,12) > 5;     % make 1 column for the output: 1 if score > 5, 0 if score <= 5

Extend the X matrix so we can treat the bias $b$ as just another weight.

In [41]:
X = [X ones(N,1)];

Let's initialize the weights to some random values

In [42]:
W = randn(12, 1)   % a column vector


W =

    2.7694
   -1.3499
    3.0349
    0.7254
   -0.0631
    0.7147
   -0.2050
   -0.1241
    1.4897
    1.4090
    1.4172
    0.6715




**Task 1**: Compute and show the cost function with the above weights

In [ ]:
%======================
% Your code here
%======================

**Task 2**: Implement optimization with Gradient Descent

You can implement a visualizaiton just like in the example provided, by copying and adapting the code.
You cannot plot all the 11 dimensions of the input data, so pick only two of them to plot.

In [ ]:
%======================
% To fill in
%======================

W = randn(12, 1);           % initialize parameters randomly

number_of_epochs = 1000;    % set number of iterations

for iter = 1:number_of_epochs
    
    % Compute predictions:
    Y-pred = ...
    
    % Compute cost:
    J(iter) = 1/N * ...
    
    % Compute derivatives according to the given formula
    dW = ...
    
    % Update the weights
    mu = 0.0001;           % try multiple values here
    W = W - mu * dW;
    
    % Store the weights history
    W_hist(:,i) = W;
end

% Plot the error and the evolution of the weights
plot(J)
figure
plot(W_hist)

**Task 3**: Compute the solution with the Matlab function `fitglm()`

In [43]:
%======================
% Your code here
%======================

## 4. Final questions

1. In our example, the parameters $W$ keep updating forever, making the gray transition area smaller and smaller, but the actual frontier does not change much. Why does this happen? How can we prevent it?

2. What happens if the two classes are **unbalanced** (many more inputs in one class compared to the other)?

2. Suggest some good termination conditions for Gradient Descent (i.e. when should we stop the iterations)?